In [1]:
from absl import app, flags, logging
from absl.flags import FLAGS
import cv2
import os

import numpy as np
import tensorflow as tf
from glob import glob

from evaluations import get_val_data, perform_val
from models import ArcFaceModel
from utils import set_memory_growth, load_yaml

In [2]:
batch_size = 128 # Initially 128
input_size = 112
embd_shape = 512
is_ccrop=False
binary_img=True

is_Adam = False   # True
dgx = True
projection_head = False  # False

head_type = 'ArcHead' # ''ArcHead', CosHead', 'SphereHead', 'CurHead', 'CadHead', 'AdaHead'
# Backbones w/ pretrained weights:
#     MobileNet, MobileNetV2, InceptionResNetV2, InceptionV3, ResNet50, ResNet50V2, ResNet101V2, NASNetLarge, NASNetMobile, Xception, MobileNetV3Large, MobileNetV3Small, EfficientNetLite0~6, EfficientNetB0~7
# Backbones w/o pretrained weights:
#      MnasNetA1, MnasNetB1, MnasNetSmall 
backbone_type = 'ResNet50' 
   
print("proj_head:", projection_head)    
print(head_type)
print(backbone_type)
print("Adam:", is_Adam)


proj_head: False
ArcHead
ResNet50
Adam: False


In [3]:
if is_Adam:
    version = "Adam"
else:
    version = "SGD"
    
if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/"
    # save_name = "ms1m_mobilenet_check/SGD/*"
    if projection_head:
        save_name = f'ms1m_{backbone_type}_{head_type}_ProjectionHead_check/{version}/*'
    else:
        save_name = f'ms1m_{backbone_type}_{head_type}_check/{version}/*'
else:
    base_dir = "/hd/honghee/models/checkpoints/w_tfidentity/"
    save_name = f'ms1m_{backbone_type}_{head_type}_check/{version}/*'   

    
## collect loss in checkpoints
file_list = []
for files in glob(f'{base_dir}{save_name}'):
    if not files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] == 'checkpoint':
        loss = float( files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] )
    file_list.append( loss  )
file_list.sort()

load_file_name = []
for files in glob(f'{base_dir}{save_name}'):
    if files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] == 'checkpoint':
        pass
    elif file_list[0] == float( files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] ) and files.split('/')[-1].split('l_')[-1].split('.ckpt')[-1]!='.index':
        load_file_name = files
best_checkpoint = load_file_name.split('.data')[0]
initial_epoch = int(load_file_name.split('e_')[-1].split('_')[0])-1
print(best_checkpoint)

AttributeError: 'list' object has no attribute 'split'

In [4]:
# best_checkpoint="/raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_ArcHead_check/Check/e_20_l_5.631781578063965.ckpt"
best_checkpoint="/home/honghee/workspace/JHFace/tmp/ckpt/ckpt-1.h5"

In [5]:
# flags.DEFINE_string('cfg_path', './configs/arc_res50.yaml', 'config file path')
# flags.DEFINE_string('gpu', '0', 'which gpu to use')
# flags.DEFINE_string('img_path', '', 'path to input image')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = '4'

logger = tf.get_logger()
logger.disabled = True
logger.setLevel(logging.FATAL)
set_memory_growth()

# cfg = load_yaml(FLAGS.cfg_path)

model = ArcFaceModel(size=input_size,
                     backbone_type=backbone_type,                     
                     training=False)

# model.load_weights(best_checkpoint)
model.load_weights(best_checkpoint, by_name=True, skip_mismatch= True)

# ckpt_path = tf.train.latest_checkpoint('./checkpoints/' + cfg['sub_name'])
# if ckpt_path is not None:
#     print("[*] load ckpt from {}".format(ckpt_path))
#     model.load_weights(ckpt_path)
# else:
#     print("[*] Cannot find ckpt from {}.".format(ckpt_path))
#     exit()

# if FLAGS.img_path:
#     print("[*] Encode {} to ./output_embeds.npy".format(FLAGS.img_path))
#     img = cv2.imread(FLAGS.img_path)
#     img = cv2.resize(img, (cfg['input_size'], cfg['input_size']))
#     img = img.astype(np.float32) / 255.
#     if len(img.shape) == 3:
#         img = np.expand_dims(img, 0)
#     embeds = l2_norm(model(img))
#     np.save('./output_embeds.npy', embeds)

print("[*] Loading LFW, AgeDB30 and CFP-FP...")
lfw, agedb_30, cfp_fp, lfw_issame, agedb_30_issame, cfp_fp_issame = get_val_data("/raid/workspace/honghee/")

print("[*] Perform Evaluation on LFW...")
acc_lfw, best_th = perform_val(
    embd_shape, batch_size, model, lfw, lfw_issame,
    is_ccrop=False)
print("    acc {:.4f}, th: {:.2f}".format(acc_lfw, best_th))

# print("[*] Perform Evaluation on AgeDB30...")
# acc_agedb30, best_th = perform_val(
#     embd_shape, batch_size, model, agedb_30,
#     agedb_30_issame, is_ccrop=False)
# print("    acc {:.4f}, th: {:.2f}".format(acc_agedb30, best_th))

# print("[*] Perform Evaluation on CFP-FP...")
# acc_cfp_fp, best_th = perform_val(
#     embd_shape, batch_size, model, cfp_fp, cfp_fp_issame,
#     is_ccrop=False)
# print("    acc {:.4f}, th: {:.2f}".format(acc_cfp_fp, best_th))

  0%|          | 0/94 [00:00<?, ?it/s]

[*] Loading LFW, AgeDB30 and CFP-FP...
[*] Perform Evaluation on LFW...


100%|██████████| 94/94 [00:20<00:00,  4.49it/s]


    acc 0.9440, th: 0.01
